## Text cleaning and spell checking

This notebook contains beginning efforts for spell checking. Formatting (conversion to integers etc.) has been moved to scraping methods.

In [1]:
import pickle, string, re
import spell

In [2]:
from enchant.checker import SpellChecker
from enchant.tokenize import EmailFilter, URLFilter
import enchant

import nltk

import ftfy
from collections import Counter

In [ ]:
from drug_objects import drug, review

In [3]:
from cleaning_refs import med_dur_conversion as med_dur_conversion
from cleaning_refs import cList as cList
new_words = open("new_words.txt", "rb").readlines()
from cleaning_refs import new_words as new_words

In [4]:
def load_pickle(_pickle_name):
    return pickle.load( open( _pickle_name, "rb" ) )

In [5]:
inverse_cList = {}
for word in cList:
    new_key = word.replace("'", "")
    inverse_cList[new_key] = cList[word]

cList_keys = cList.keys()
inverse_cList_keys = inverse_cList.keys()

In [46]:
def expandContractions(text, words):#, c_re=c_re):
    c_re = re.compile('(%s)' % '|'.join(words.keys()))
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [42]:
#method of review?
def metaData_clean(_reviews):
    for ik, _review in enumerate(_reviews):
        #pull out contractions
        comment_text = (_review.comment).split()
        for word in comment_text:
            if len(word.split("'s"))>1:
                word = word.split("'s")[0]
            if word in cList_keys:
                word= cList[word]
            if word in inverse_cList_keys:
                word = inverse_cList[word]

        _review.comment = ' '.join(comment_text)
        
        #remove punctuation
        remove = string.punctuation
        remove = remove.replace("-", "") # don't remove hyphens
        pattern = r"[{}]".format(remove) 
        _review.comment = re.sub(pattern, "",  _review.comment) 

    return _reviews

In [34]:
d2 = enchant.DictWithPWL("en_US","new_words.txt")
chkr = SpellChecker("en_US", filters=[EmailFilter,URLFilter])

def spellchecker(reviews):
    for ik, review in enumerate(reviews):
            chkr.set_text(review.comment)
            for err in chkr:
                # forFuture: add catch for repeated character, e.g. 'soooooooo'
                error = (err.word).lower()
                options = [change.lower() for change in d2.suggest(error)]
                if error in options:
                    continue
                else:
                    catch = 'no'
                    word_to_beat = options[0]
                    score_to_beat = 10
                    for word in options:
                        dist = nltk.edit_distance(error, word)
                        if (word in new_words):
                            if (nltk.edit_distance(error, word)<=score_to_beat):
                                word_to_beat = word
                                score_to_beat = nltk.edit_distance(error, word)
                                catch = 'yes'
                        elif dist<score_to_beat:
                            word_to_beat = word
                            score_to_beat = nltk.edit_distance(error, word)
                    print('final option:', word_to_beat, ',\t\t error:', error, 'score:', score_to_beat)

In [19]:
pickle_name = "drug_list_ddc_pg1.p"

In [20]:
drug_list = load_pickle(pickle_name)
print(len(drug_list[0].reviews))

drug_list_ddc.p


In [ ]:
drug_list[1].__dict__

In [43]:
metaData_clean(drug_list[0].reviews)

None
[1, 6]
['I', 'was', 'taking', 'Cymbalta', 'for', '2', '12', 'months', 'along', 'with', 'Lamictal', 'which', 'I', 'still', 'take', 'to', 'get', 'rid', 'of', 'the', 'depressive', 'side', 'of', 'bipolar', 'It', 'made', 'me', 'super', 'down', 'and', 'extremely', 'on', 'edge', 'for', 'weeks', 'I', 'was', 'on', '20mg']
None
None
None
[0, 1]
['I', 'took', 'my', 'first', 'dose', 'of', 'generic', 'Cymbalta', 'last', 'night', 'of', '60', 'milligrams', 'Its', 'now', '547', 'PM', 'Ive', 'not', 'eaten', 'all', 'day', 'due', 'to', 'the', 'severe', 'nausea', 'Im', 'also', 'very', 'jittery', 'anxious', 'have', 'a', 'super', 'really', 'bad', 'headache', 'Im', 'also', 'prone', 'to', 'migraines', 'my', 'tinnitus', 'is', 'worse', 'in', 'my', 'left', 'ear', 'I', 'also', 'have', 'slept', 'most', 'of', 'the', 'day', 'besides', 'I', 'have', 'depression', 'mild', 'PTSD', 'anxiety', 'migraines', 'Ive', 'been', 'on', 'other', 'anti-depressants', 'before', 'Prozac', 'increased', 'my', 'migraines', 'the', 'la

In [55]:
spellchecker(reviews)

final option: realized ,		 error: realised score: 1
final option: prissy ,		 error: pissy score: 1
final option: cu ,		 error: cuz score: 1
final option: kingliness ,		 error: tingliness score: 1
final option: col ,		 error: coli score: 1
final option: lamictal ,		 error: lamactal score: 1
final option: sooths ,		 error: sooooo score: 3
final option: fibroid ,		 error: fibro score: 2
final option: copyable ,		 error: copable score: 1
final option: withdrawing ,		 error: wothdrawaling score: 3
final option: bonzes ,		 error: benzos score: 2
final option: trazodone ,		 error: trazadone score: 1
final option: prescription ,		 error: perscription score: 2
final option: dot ,		 error: doct score: 1
final option: physician ,		 error: physican score: 1
final option: recommended ,		 error: recommeded score: 1
final option: physician ,		 error: physican score: 1
final option: ms ,		 error: mds score: 1
final option: want ,		 error: wasnt score: 1
final option: want ,		 error: wasnt score: 1
fin